In [7]:
# Make sure to install these libraries
# pip install ipywidgets
# jupyter nbextension enable

In [8]:
# import libraries
import pandas as pd
import statsmodels.api as sm
from ipywidgets import interact, SelectMultiple

In [9]:
# Load dataset
df = pd.read_csv("data/df_enriched.csv")

target_var = "life_expectancy"

candidate_predictors = [
    "health_expenditure_usd",
    "physicians_per_1000",
    "nurses_per_1000",
    "smoking_prevalence",
    "diabetes_prevalence",
    "pollution_mortality_rate"
]

In [10]:
import pandas as pd
import statsmodels.api as sm
from ipywidgets import interact, SelectMultiple

# Load dataset
df = pd.read_csv("data/df_enriched.csv")

target_var = "life_expectancy"

candidate_predictors = [
    "health_expenditure_usd",
    "physicians_per_1000",
    "nurses_per_1000",
    "smoking_prevalence",
    "diabetes_prevalence",
    "pollution_mortality_rate",
]

@interact(
    predictors=SelectMultiple(
        options=candidate_predictors,
        value=("health_expenditure_usd",),   # default selection
        description="Select predictors",
    )
)
def run_regression(predictors):
    # Convert tuple -> list so pandas understands it
    predictors = list(predictors)

    if len(predictors) == 0:
        print("⚠ Please select at least ONE independent variable.")
        return

    # Build modeling DataFrame and drop rows with missing values
    cols = [target_var] + predictors
    df_model = df[cols].dropna()

    if df_model.empty:
        print("⚠ No usable rows after filtering — try fewer predictors.")
        return

    X = df_model[predictors]
    y = df_model[target_var]

    # Add intercept and fit OLS
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    print("================================")
    print(f"Dependent variable: {target_var}")
    print(f"Predictors: {', '.join(predictors)}")
    print(f"Observations used: {len(df_model)}")
    print(f"R² = {model.rsquared:.3f}")
    print("================================")
    print(model.summary())


interactive(children=(SelectMultiple(description='Select predictors', index=(0,), options=('health_expenditure…